In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score



from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D,merge, MaxPooling1D, Embedding,Merge,  Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.merge import Add,Concatenate,Dot


BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100
num_words = MAX_NB_WORDS
MOD_DIR = "/home/abhinav/data/sixth_model_alan_filtering/"


print('Loading data...')
[x_train, y_train] = pd.read_pickle(MOD_DIR + "train_data") 

[x_test, y_test] = pd.read_pickle(MOD_DIR + "test_dataset") 

print("Loading the Embedding matrix")

embedding_matrix = pd.read_pickle(MOD_DIR + "embedding_matrix")



#some helper functions

class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = self.model.predict(self.validation_data[0],batch_size = 512)
        predict = prob_to_label(predict)
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("\nWeighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

def prob_to_label(array):
    for a in array:
        if a[0] > a[1] :
            a[0] = 1
            a[1] = 0
        elif a[0] < a[1]:
            a[0] = 0
            a[1] = 1
        else:
            ind = np.random.randint(2)
            a[ind]= 1
            a[int(1-ind)] = 0
    return array



Using TensorFlow backend.


Loading data...
Loading the Embedding matrix


In [8]:



#Declaring the Model


embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
l_flat = Flatten()(l_pool1)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)


model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_7 (Embedding)          (None, 1000, 100)     5000000     input_7[0][0]                    
____________________________________________________________________________________________________
conv1d_22 (Conv1D)               (None, 998, 32)       9632        embedding_7[0][0]                
____________________________________________________________________________________________________
conv1d_23 (Conv1D)               (None, 997, 32)       12832       embedding_7[0][0]                
___________________________________________________________________________________________

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [9]:
initial_weights = model.get_weights()


In [ ]:
model.set_weights(initial_weights)
print('Train...')
metrics = Metrics(MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 8},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [10]:
model.set_weights(initial_weights)
print('Train...')
metrics = Metrics(MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 10},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train...
Train on 9937 samples, validate on 45500 samples
Epoch 1/10
9856/9937 [============================>.] - ETA: 0s - loss: 1.2842 - acc: 0.4961

KeyboardInterrupt: 

In [ ]:
model.set_weights(initial_weights)
print('Train...')
metrics = Metrics(MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
print('Train...')
metrics = Metrics(MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 15},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
print('Train...')
metrics = Metrics(MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 5},
validation_data=[x_test, y_test],
         callbacks = [metrics])